In [1]:
import os
from pathlib import Path
import sys
import warnings

import numpy as np
import pandas as pd
from mizani.formatters import percent_format
from plotnine import *
from scipy.stats import logistic
from scipy.stats import norm
from stargazer.stargazer import Stargazer
from patsy import dmatrices
import statsmodels.api as sm
import statsmodels.formula.api as smf
from statsmodels.iolib.summary2 import summary_col
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score
from sklearn.metrics import log_loss

import plotly
import plotly.express as px
import pandas as pd
import plotly.io as pio
import plotly.graph_objects as go
from plotly.subplots import make_subplots
pio.templates.default = 'plotly'

warnings.filterwarnings("ignore")

In [2]:
path = Path(os.getcwd())
tech_prep = os.path.join(str(path), "utils/")
sys.path.append(tech_prep)
from py_helper_functions import *

In [3]:
# Loading data and checking
main_df = pd.read_csv('https://osf.io/4ay9x/download')
main_df.head().T

,0,1,2,3,4
Unnamed: 0,3,5,6,10,11
hhid,2600310997690,75680310997590,75680310997590,179140131100930,179140131100930
intmonth,January,January,January,January,January
stfips,AL,AL,AL,AL,AL
weight,3151.6801,3457.1138,3936.911,3288.364,3422.85
earnwke,1692.0,450.0,1090.0,769.23,826.92
uhours,40,40,60,40,40
grade92,43,41,41,40,43
race,1,2,2,1,1
ethnic,NaN,NaN,NaN,NaN,NaN


In [14]:
df = main_df.loc[main_df['occ2012'] == 2310] # Filtering for Secretaries and administrative assistants

In [15]:
df.describe()

,Unnamed: 0,hhid,weight,earnwke,uhours,grade92,race,ethnic,age,sex,marital,ownchild,chldpres,occ2012
count,3636.000000,3.636000e+03,3636.000000,3636.000000,3636.000000,3636.000000,3636.000000,261.000000,3636.000000,3636.000000,3636.000000,3636.000000,3636.000000,3636.0
mean,159809.939769,4.480944e+14,2317.608004,1034.200594,40.691144,43.308306,1.277778,2.567050,42.160066,1.820132,2.536304,0.883938,2.360286,2310.0
std,92380.409835,3.202843e+14,1253.577145,528.825998,8.737564,1.129496,1.240695,2.385918,11.163914,0.384130,2.443091,1.104078,3.301203,0.0
min,43.000000,8.171510e+09,65.994300,0.230000,1.000000,34.000000,1.000000,1.000000,16.000000,1.000000,1.000000,0.000000,0.000000,2310.0
25%,79668.250000,1.408828e+14,1217.922700,692.300000,40.000000,43.000000,1.000000,1.000000,33.000000,2.000000,1.000000,0.000000,0.000000,2310.0
50%,161146.500000,4.103550e+14,2655.192200,961.000000,40.000000,43.000000,1.000000,1.000000,42.000000,2.000000,1.000000,0.000000,0.000000,2310.0
75%,241507.500000,7.219705e+14,3279.228425,1250.000000,40.000000,44.000000,1.000000,3.000000,51.000000,2.000000,5.000000,2.000000,4.000000,2310.0
max,317003.000000,9.998102e+14,10672.155900,2884.610000,80.000000,46.000000,21.000000,8.000000,64.000000,2.000000,7.000000,9.000000,15.000000,2310.0


In [16]:
df = df.loc[(main_df['uhours'] >= 20) # Filtering for at least 20 hours/week worked 
                 & (main_df['age'] >= 18) # Filtering for at least 18 years of age
                 & (main_df['earnwke'] > 0) # Filtering for more than 0 wage
                ]                     
df.shape

(3537, 23)

In [17]:
# Creating our target variable earnings per hour 'eph'

df['eph'] = df['earnwke'] / df['uhours']
df['eph'].describe()

count    3537.000000
mean       25.572499
std        12.586661
min         0.004107
25%        17.094000
50%        23.076750
75%        31.250000
max       100.125000
Name: eph, dtype: float64

## Exploring variables and creating dummies 
    1. Education level
    2. Age
    3. Gender
    4. Married
    5. has children
    6. Union
    7. Private or Public
    8. Race

In [21]:
# Exploring education leve
df.describe().round(2)

,Unnamed: 0,hhid,weight,earnwke,uhours,grade92,race,ethnic,age,sex,marital,ownchild,chldpres,occ2012,eph
count,3537.00,3.537000e+03,3537.00,3537.00,3537.00,3537.00,3537.00,256.00,3537.00,3537.00,3537.00,3537.00,3537.00,3537.0,3537.00
mean,159844.92,4.468632e+14,2317.60,1054.05,41.48,43.33,1.27,2.50,42.08,1.82,2.55,0.87,2.34,2310.0,25.57
std,92145.98,3.199704e+14,1256.61,518.83,7.41,1.10,1.21,2.34,11.12,0.39,2.45,1.09,3.29,0.0,12.59
min,43.00,8.171510e+09,65.99,0.23,20.00,34.00,1.00,1.00,18.00,1.00,1.00,0.00,0.00,2310.0,0.00
25%,79937.00,1.407641e+14,1217.92,711.53,40.00,43.00,1.00,1.00,33.00,2.00,1.00,0.00,0.00,2310.0,17.09
50%,161147.00,4.100094e+14,2649.98,961.53,40.00,43.00,1.00,1.00,42.00,2.00,1.00,0.00,0.00,2310.0,23.08
75%,241005.00,7.207065e+14,3283.37,1269.23,40.00,44.00,1.00,3.00,51.00,2.00,5.00,2.00,4.00,2310.0,31.25
max,317003.00,9.998102e+14,10672.16,2884.61,80.00,46.00,21.00,8.00,64.00,2.00,7.00,9.00,15.00,2310.0,100.12


In [32]:
df.groupby('grade92')['eph'].describe()

,count,mean,std,min,25%,50%,75%,max
grade92,,,,,,,,
34,1.0,6.666667,NaN,6.666667,6.666667,6.666667,6.666667,6.666667
36,1.0,10.075000,NaN,10.075000,10.075000,10.075000,10.075000,10.075000
37,3.0,9.777778,1.347151,8.333333,9.166667,10.000000,10.500000,11.000000
38,6.0,10.475000,8.934414,0.750000,8.125000,8.500000,9.325000,27.500000
39,66.0,17.184219,10.571165,2.884500,9.406250,14.520000,22.562500,60.000000
40,90.0,17.385124,9.797766,3.296571,11.052632,14.900000,21.692250,60.250000
41,29.0,15.346925,7.987571,6.250000,10.000000,14.300000,16.567143,42.000000
42,65.0,18.988439,13.327443,1.922917,9.615250,15.000000,22.000000,72.115250
43,1518.0,23.893208,12.433260,0.004107,15.821333,20.769200,28.825000,100.125000


In [ ]:
df['HS_GED'] = (df['grade92'] == 39).astype(int) # Associate degree (Vocational/occupational)
df['college_dropout'] = (df['grade92'] == 40).astype(int) # Associate degree (Vocational/occupational)
df['AD_V'] = (df['grade92'] == 41).astype(int) # Associate degree (Vocational/occupational)
df['AD_AP'] = (df['grade92'] == 42).astype(int) # Associate degree (Academic Program)
df['BD'] = (df['grade92'] == 43).astype(int) # Bachelor's degree (e.g.BA,AB,BS)
df['MD'] = (df['grade92'] == 44).astype(int) # Master's degree (e.g.MA,MS,MEng,Med,MSW,MBA) 
df['PD'] = (df['grade92'] == 45).astype(int) # Professional degree (e.g.MD,DDS,DVM,LLB,JD)
df['PhD'] = (df['grade92'] == 46).astype(int) # Doctorate degree(e.g.PhD,EdD)

In [28]:
df['female'] = np.where(df['sex'] == 2, '1', '0')
df['female'].value_counts()

female
1    2890
0     647
Name: count, dtype: int64

In [35]:
df['married'] = np.where(df['marital'] <=2, '1', '0')
df['separated'] = np.where((df['marital'] >= 3) & (df['marital'] <= 6), '1', '0')
df['never_married'] = np.where(df['marital'] == 7, '1', '0')


In [40]:
df['has_children'] = np.where(df['ownchild'] > 0, '1', '0')

In [42]:
df['is_union_member'] = np.where(df['unionmme'] == 'Yes', '1', '0')

In [45]:
df['class'].value_counts()

class
Government - Local      2122
Government - State       689
Private, For Profit      428
Private, Nonprofit       276
Government - Federal      22
Name: count, dtype: int64

In [53]:
df['government_employee'] = np.where((df['class'] == 'Government - Federal')
                                    |(df['class'] == 'Government - Local')
                                    |(df['class'] == 'Government - State'), '1', '0')

df['private_employee'] = np.where((df['class'] == 'Private, For Profit')
                                    |(df['class'] == 'Private, Nonprofit'), '1', '0')

prcitshp
Native, Born In US                        3348
Foreign Born, US Cit By Naturalization     102
Foreign Born, Not a US Citizen              49
Native, Born Abroad Of US Parent(s)         30
Native, Born in PR or US Outlying Area       8
Name: count, dtype: int64

In [59]:
df['race'].value_counts()

race
1     3100
2      292
4       73
3       19
5       13
7        8
6        7
8        6
9        5
16       4
21       3
15       3
11       2
10       1
13       1
Name: count, dtype: int64

In [60]:
df['white_person'] = np.where(df['race'] == 1, '1', '0')
df['non_white_person'] = np.where(df['race'] > 1 , '1', '0')